## Core

- Measure fluxes from a region at multiple frequencies



In [4]:
import numpy as np
from astropy.io import ascii, fits
import fluxInt
import pbCorr
import synCl
import convRegr
from prettytable import PrettyTable
import shutil,os, string
import fitSource


ftSrc = fitSource.fitSrc()
fint = fluxInt.flInt()
cvReg = convRegr.cvRegr()
pb = pbCorr.pbcorr()
syn = synCl.synClean()

#rootdir
#rootDir = '/Users/maccagni/Projects/MFS/FornaxA/contData/kore/'
rootDir = '/home/maccagni/Projects/MFS/FornaxA/contData/core/'

#label

aut='Pbcorr'
#aut='PbcorrCv'


#fluxIntOpt = 'RgCv'
fluxIntOpt = None

label = ['Tot','Beam','Out']
synMod = ['CI', 'CIOFF']

inTabs = rootDir+'inTabs/'
imDir = rootDir+'inIms/'
tmpDir = rootDir+'tmpIms/'
synIn = rootDir+'synIn/'
synOut = rootDir+'synOut/'
regDir = imDir+'inReg/'
outTabs = rootDir+'outTabs/'

#FLUX Tables
coreTot = outTabs+'coreFlux'+label[0]+aut+'.tbl'
outBeam=outTabs+'coreFlux'+label[1]+aut+'.tbl'
outOut=outTabs+'coreFlux'+label[2]+aut+'.tbl'

synageTot = synIn+'synageFlux'+label[0]+aut+'.tbl'
synageBeam = synIn+'synageFlux'+label[1]+aut+'.tbl'
synageOut = synIn+'synageFlux'+label[2]+aut+'.tbl'

#Regions where to measure flux
region = regDir+'cReg.reg'
regAlma = regDir+'cRegAL.reg'

fint.rootdir = imDir
syn.rootdir = imDir
pb.rootdir = imDir

tableName = inTabs+'coreFileNames'+aut+'.tbl'
tableFileNames= ascii.read(tableName,format='csv')

print tableFileNames.columns[0]

print '''\t+---------+\n\t Inputs loaded\n\t+---------+'''


                                    FileName                                    
--------------------------------------------------------------------------------
/home/maccagni/Projects/MFS/FornaxA/contData/core/inIms/Meer_1030MHz_pbcorr.fits
/home/maccagni/Projects/MFS/FornaxA/contData/core/inIms/Meer_1440MHz_pbcorr.fits
 /home/maccagni/Projects/MFS/FornaxA/contData/core/inIms/VLA_4860MHz_pbcorr.fits
   /home/maccagni/Projects/MFS/FornaxA/contData/core/inIms/VLA_15GHz_pbcorr.fits
        /home/maccagni/Projects/MFS/FornaxA/contData/core/inIms/ALMA_allGHz.fits
	+---------+
	 Inputs loaded
	+---------+


- We measure the fluxes following 2 different methods
    - Total flux of innermost 1.5 kpc. 
    - Total flux of the central beam and of the outer regions
    
(Total flux is measured setting a cutoff at $3\sigma$ the noise level).

We measure the flux in 6 frequency interval centred at: 

- 1.030 GHz (MeerKAT)
- 1.440 GHz (MeerKAT)
- 1.500 GHz (VLA)
- 4.800 GHz (VLA)
- 14.00 GHz (VLA)
- 108.0 GHz (ALMA)


We measure the flux in each map at its original spatial resolution. Since the images have very different fields of view, we draw a broad region in the centre that includes the core in all images. We measure as flux all signal above 3$\cdot$ r.m.s of the image.

The noise of the MeerKAT images (and VLA at 1500 MHz) is given in the input table, since it needs to be measured outside the extended radio lobes, that otherwise would be included in the measurement.


In [ ]:
#-------------------------------------------------#
# Total flux of innermost 1.5 kpc
#-------------------------------------------------#
noise = tableFileNames.columns[2]
errFlux =tableFileNames.columns[3]

fr=[]
flIntArray=[]
flErrArray=[]

for i in xrange(0,len(tableFileNames.columns[0])):

    fileName =tableFileNames.columns[0][i]
    datas,heads = fint.openFile(fileName)

    maskedData, background, rms, pixels=fint.maskDatReg(datas,heads,region,noise[i])

    noiseInt = rms *np.sqrt(pixels)
    
    if 'Frequency' in tableFileNames.dtype.names:
        freq = float(tableFileNames.columns[1][i])*1e6
        fr.append(freq)
                
    fluxint, numPixBeam =fint.measFlux(maskedData,heads,errFlux,'core')
    flIntArray.append(fluxint)
    
    t,flErr = fint.writeTable(heads,fluxint,noiseInt,numPixBeam,freq,errFlux[i],coreTot)
    flErrArray.append(flErr)

syn.writeSynageTable(synageTot,fr,flIntArray,flErrArray,'Core')


#os.rename(synageFlTot,synageTotFluxTable)
print '''\t+---------+\n\t Fluxes measured\n\t+---------+'''
  

In [ ]:
#-------------------------------------------------#
# Total flux of central beam and of outer region
#-------------------------------------------------#


noise = tableFileNames.columns[2]
errFlux =tableFileNames.columns[3]


if os.path.exists(rootDir+outBeam):
    os.remove(rootDir+outBeam)
if os.path.exists(rootDir+outOut):
    os.remove(rootDir+outOut)
fr=[]
flIntBeamArray=[]
flErrBeamArray=[]
flIntOutArray=[]
flErrOutArray=[]

for i in xrange(0,len(tableFileNames.columns[0])):

    fileName = tableFileNames.columns[0][i]
    print string.split(fileName,'/')[-1]
    datas,heads = fint.openFile(fileName)

    maskedData, background, rms, pixels=fint.maskDatReg(datas,heads,region,noise[i])
    #fits.writeto(rootDir+str(i)+'_mask.fits',maskedData,heads,overwrite=True)

    pixelsOut = np.count_nonzero(~np.isnan(maskedData))
   
    maskedData[np.isnan(maskedData)] = 0.0
    
    params = ftSrc.gaus2Dfit(maskedData)

    peak =  params[0]
    
    peak = np.nanmax(maskedData)
    
    maskedData[maskedData==0.0] = np.nan

    fluxint, numPixBeam =fint.measFlux(maskedData,heads,errFlux,fluxIntOpt)

    
    #maskedData[~np.isnan(maskedData)] = 1.
    fits.writeto(tmpDir+str(i)+'_mask.fits',maskedData,heads,overwrite=True)
    
    fluxintOut = fluxint - peak
    print 'Core\tJets'
    print fluxint
    print str(peak)+'\t'+str(fluxintOut)+'\n'

    if 'Frequency' in tableFileNames.dtype.names:
        freq = float(tableFileNames.columns[1][i])*1e6
        fr.append(freq)
                
    noiseIntBeam = noise[i]/numPixBeam
    
    flIntBeamArray.append(peak)

    t,flBeamErr = fint.writeTable(heads,peak,noiseIntBeam,numPixBeam,freq,errFlux[i],outBeam)
    
    flErrBeamArray.append(flBeamErr)    
    flIntOutArray.append(fluxintOut)

    noiseIntOut = noise[i]/numPixBeam*np.sqrt(pixelsOut*numPixBeam)

    t,flOutErr = fint.writeTable(heads,fluxintOut,noiseIntOut,numPixBeam,freq,errFlux[i],outOut)
    
    flErrOutArray.append(flOutErr)

    
syn.writeSynageTable(synageBeam,fr,flIntBeamArray,flErrBeamArray,'Beam'+aut)
syn.writeSynageTable(synageOut,fr,flIntOutArray,flErrOutArray,'Out'+aut)


print '''\t+---------+\n\t Fluxes measured\n\t+---------+'''
  